## 1.1 Load Department of Education (DOE) and CEDA

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load Department of Education data
DOE_data = pd.read_excel('/content/drive/My Drive/Candidate Platforms and Endorsements/Three files discussed 10-18-23/Base_File_from_State_Dept_Ed.xls')
DOE_data.head()

,County,Year,DOCType,District,Status,WebSite,cds7,leaid,DOC
0,Alameda,1996,Unified School District,Alameda Unified,Active,http://www.alamedaunified.org,161119,601770,54
1,Alameda,1997,Unified School District,Alameda Unified,Active,http://www.alamedaunified.org,161119,601770,54
2,Alameda,1998,Unified School District,Alameda Unified,Active,http://www.alamedaunified.org,161119,601770,54
3,Alameda,1999,Unified School District,Alameda Unified,Active,http://www.alamedaunified.org,161119,601770,54
4,Alameda,2000,Unified School District,Alameda Unified,Active,http://www.alamedaunified.org,161119,601770,54


In [ ]:
# Load truncated unique district-year CEDA elections
CEDA_data = pd.read_excel('/content/drive/My Drive/Candidate Platforms and Endorsements/Three files discussed 10-18-23/All unique district-year elections from CEDA.xlsx')
CEDA_data.head()

,CNTYNAME,YEAR,PLACE,election_type,countyboard,coe,leaid
0,ALAMEDA,1996,ALAMEDA BOARD OF EDUCATION,Regular,NaN,NaN,NaN
1,ALAMEDA,2000,ALAMEDA CITY UNIFIED,Regular,NaN,NaN,NaN
2,ALAMEDA,2004,ALAMEDA CITY UNIFIED,Regular,NaN,NaN,NaN
3,ALAMEDA,2006,ALAMEDA CITY UNIFIED,Regular,NaN,NaN,NaN
4,ALAMEDA,2010,ALAMEDA CITY UNIFIED,Regular,NaN,NaN,NaN


In [ ]:
CEDA_data = CEDA_data[(CEDA_data['countyboard'] != 1) & (CEDA_data['coe'] != 1)]
CEDA_data.drop(['countyboard', 'coe'], axis=1)

,CNTYNAME,YEAR,PLACE,election_type,leaid
0,ALAMEDA,1996,ALAMEDA BOARD OF EDUCATION,Regular,NaN
1,ALAMEDA,2000,ALAMEDA CITY UNIFIED,Regular,NaN
2,ALAMEDA,2004,ALAMEDA CITY UNIFIED,Regular,NaN
3,ALAMEDA,2006,ALAMEDA CITY UNIFIED,Regular,NaN
4,ALAMEDA,2010,ALAMEDA CITY UNIFIED,Regular,NaN
...,...,...,...,...,...
8357,YUBA,2012,WHEATLAND UNION HIGH,Regular,NaN
8358,YUBA,2014,WHEATLAND UNION HIGH,Regular,NaN
8359,YUBA,2016,WHEATLAND UNION HIGH,Regular,NaN
8360,YUBA,2020,WHEATLAND UNION HIGH,Regular,NaN


In [ ]:
# Standardize column names
CEDA_data.columns = [x.lower() for x in CEDA_data.columns]
DOE_data.columns = [x.lower() for x in DOE_data.columns]

# Standardize strings
CEDA_data['cntyname'] = CEDA_data['cntyname'].str.lower()
CEDA_data['place'] = CEDA_data['place'].str.lower()

DOE_data['county'] = DOE_data['county'].str.lower()
DOE_data['district'] = DOE_data['district'].str.lower()

In [ ]:
# Check county lists between DOE/CEDA, conclude identical
ceda_counties = CEDA_data['cntyname'].unique()
doe_counties = DOE_data['county'].unique()

set1 = set(ceda_counties)
set2 = set(doe_counties)

unique_elements_in_array1 = set1.difference(set2)
unique_elements_in_array2 = set2.difference(set1)

ceda_counties_diff = list(unique_elements_in_array1)
doe_counties_diff = list(unique_elements_in_array2)

print(ceda_counties_diff)
print(doe_counties_diff)

[]
[]


## 1.2 Match from DOE to CEDA

In [ ]:
!pip install fuzzywuzzy

In [ ]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
# Get word counts in all names
appended = ''

for i, row in DOE_data.iterrows():
  district = row['district']
  appended = appended + ' ' + district

counts = {}

words = appended.split()
uniq_words = set(words)
for word in uniq_words:
  counts[word] = appended.count(word)

# Get words to remove from keywords
sorted_counts = dict(sorted(counts.items(), key=lambda item: item[1], reverse=True))
filler = {'elementary', 'union', 'unified', 'high', 'joint', 'los', 'valley', 'san', 'city', 'beach'}

In [ ]:
sorted_counts

{'le': 22977,
 'el': 19602,
 'elem': 16956,
 'eleme': 16902,
 'elemen': 16821,
 'element': 16794,
 'elementa': 16767,
 'elementar': 16740,
 'elementary': 16659,
 'union': 10827,
 'unified': 9450,
 'la': 3915,
 'hig': 3267,
 'high': 3240,
 'joint': 2835,
 'val': 2241,
 'valle': 2133,
 'valley': 2025,
 'san': 1944,
 '-': 1755,
 'city': 1350,
 'lake': 702,
 'di': 675,
 'mar': 540,
 'oak': 540,
 'santa': 540,
 'los': 486,
 'del': 432,
 'fe': 432,
 'side': 432,
 'river': 405,
 'creek': 405,
 'view': 378,
 'the': 378,
 'hill': 351,
 'south': 351,
 'west': 351,
 'ana': 351,
 'mount': 324,
 'park': 324,
 'oro': 297,
 'beach': 297,
 'spring': 297,
 'old': 297,
 'grove': 297,
 'new': 270,
 'mountain': 270,
 'bar': 270,
 'monte': 243,
 'pleasant': 243,
 'big': 243,
 'county': 243,
 'hart': 243,
 'north': 216,
 'sun': 216,
 'red': 216,
 'hills': 189,
 'harte': 189,
 'mad': 189,
 'palo': 189,
 'gold': 189,
 'fort': 189,
 'ridge': 189,
 'rio': 189,
 'vista': 189,
 'center': 189,
 'pine': 189,
 'las'

In [ ]:
# Get districts that might get filtered out
for i, row in DOE_data.iterrows():
  district = row['district']
  words = district.split()
  if words[0] in filler:
    print(district)

san leandro unified
san leandro unified
san leandro unified
san leandro unified
san leandro unified
san leandro unified
san leandro unified
san leandro unified
san leandro unified
san leandro unified
san leandro unified
san leandro unified
san leandro unified
san leandro unified
san leandro unified
san leandro unified
san leandro unified
san leandro unified
san leandro unified
san leandro unified
san leandro unified
san leandro unified
san leandro unified
san leandro unified
san leandro unified
san leandro unified
san leandro unified
san lorenzo unified
san lorenzo unified
san lorenzo unified
san lorenzo unified
san lorenzo unified
san lorenzo unified
san lorenzo unified
san lorenzo unified
san lorenzo unified
san lorenzo unified
san lorenzo unified
san lorenzo unified
san lorenzo unified
san lorenzo unified
san lorenzo unified
san lorenzo unified
san lorenzo unified
san lorenzo unified
san lorenzo unified
san lorenzo unified
san lorenzo unified
san lorenzo unified
san lorenzo unified


In [ ]:
matches = []

election_type = []

# Keep track of CEDA that were matched
CEDA_matched = {}

# Iterate through DOE data
for i, row in DOE_data.iterrows():
  year = row['year']
  county = row['county']
  district = row['district']
  candidates = list(CEDA_data[(CEDA_data['year'] == year) & (CEDA_data['cntyname'] == county)]['place'])

  # Dictionary for filtered: original name
  cand_dict = {}

  # Extract keywords in district name
  words = district.split()
  distr_word_list = [words[0]]
  for i in range(1, len(words)):
    if words[i] not in filler:
      distr_word_list.append(words[i])

  # Join keywords in district
  cleaned_distr = " ".join(distr_word_list)

  # Extract keywords in candidates
  filtered = []
  for candidate in candidates:
    words = candidate.split()
    cand_word_list = []

    # Extract keywords in candidates
    for i in range(len(words)):
      if words[i] not in filler:
        cand_word_list.append(words[i])

    # Join keywords in candidates
    cleaned_cand = ' '.join(cand_word_list)

    # Cleaned: original candidate
    cand_dict[cleaned_cand] = candidate

    # Consider candidate only if have common keyword
    for keyword in distr_word_list:
      if keyword in cand_word_list:
        filtered.append(cleaned_cand)
        break
  candidates = filtered

  if len(candidates) == 0:
    matches.append(['none', 0])
    election_type.append('none')
  else:
    match, score = process.extractOne(district, candidates, scorer=fuzz.token_set_ratio)
    matches.append([cand_dict[match], score])
    election_type.append(CEDA_data[(CEDA_data['year'] == year) & (CEDA_data['cntyname'] == county) & (CEDA_data['place'] == cand_dict[match])]['election_type'].item())
    if (cand_dict[match], year, county) in CEDA_matched:
      CEDA_matched[(cand_dict[match], year, county)].append(district)
    elif (cand_dict[match], year, county) not in CEDA_matched:
      CEDA_matched[(cand_dict[match], year, county)] = [district]

In [ ]:
len(election_type)

32022

In [ ]:
# Append matches into columns on dataframe
DOE_data['match'] = [elem[0] for elem in matches]
DOE_data['score'] = [elem[1] for elem in matches]
DOE_data['election type'] = election_type
DOE_data

,county,year,doctype,district,status,website,cds7,leaid,doc,match,score,election type
0,alameda,1996,Unified School District,alameda unified,Active,http://www.alamedaunified.org,161119,601770,54,alameda board of education,64,Regular
1,alameda,1997,Unified School District,alameda unified,Active,http://www.alamedaunified.org,161119,601770,54,none,0,none
2,alameda,1998,Unified School District,alameda unified,Active,http://www.alamedaunified.org,161119,601770,54,alameda co. office of education,64,Regular
3,alameda,1999,Unified School District,alameda unified,Active,http://www.alamedaunified.org,161119,601770,54,none,0,none
4,alameda,2000,Unified School District,alameda unified,Active,http://www.alamedaunified.org,161119,601770,54,alameda city unified,100,Regular
...,...,...,...,...,...,...,...,...,...,...,...,...
32017,yuba,2018,High School District,wheatland union high,Active,http://www.wheatlandhigh.org,5872769,642350,56,wheatland elementary,100,Regular
32018,yuba,2019,High School District,wheatland union high,Active,http://www.wheatlandhigh.org,5872769,642350,56,none,0,none
32019,yuba,2020,High School District,wheatland union high,Active,http://www.wheatlandhigh.org,5872769,642350,56,wheatland union high,100,Regular
32020,yuba,2021,High School District,wheatland union high,Active,http://www.wheatlandhigh.org,5872769,642350,56,none,0,none


In [ ]:
# District : [Number of even year elections, total number of elections]
elections_2015 = {}

# Calculuate proportion of odd/even year elections before 2015
for i, row in DOE_data.iterrows():
  if row['match'] != 'none' and row['year'] <= 2015:
    if row['district'] in elections_2015:
      if row['year'] % 2 == 0:
        elections_2015[row['district']][0] += 1
      elections_2015[row['district']][1] += 1
    elif row['district'] not in elections_2015:
      if row['year'] % 2 == 0:
        elections_2015[row['district']] = [1, 1]
      else:
        elections_2015[row['district']] = [0, 1]

even_proportions = []
# Display proportions of even year elections before 2015
for i, row in DOE_data.iterrows():
  if row['district'] in elections_2015:
    even_proportion = elections_2015[row['district']][0]/elections_2015[row['district']][1]
    even_proportions.append(even_proportion)
  else:
    even_proportions.append(0)

DOE_data['even proportion before 2015'] = even_proportions

In [ ]:
DOE_data.to_excel('/content/drive/MyDrive/Candidate Platforms and Endorsements/Data files/merged_DOE-CEDA.xlsx', index=False)

In [ ]:
print('Number of entries that got matched in CEDA:', len(CEDA_matched))
print('Total number of CEDA entries:', len(CEDA_data))

Number of entries that got matched in CEDA: 6687
Total number of CEDA entries: 7873


In [ ]:
# Find which CEDA entries did get matched/not matched
matched = []
which_matched = []

counties = {}
multi_county = []
which_counties = []

# Get matches and counties
for i, row in CEDA_data.iterrows():
  county = row['cntyname']
  year = row['year']
  district = row['place']

  # Keep track of matches
  if (district, year, county) in CEDA_matched:
    matched.append(1)
    which_matched.append(CEDA_matched[(district, year, county)])
  else:
    matched.append(0)
    which_matched.append([])

  # Keep track of counties
  if district in counties:
    counties[district].add(county)
  else:
    counties[district] = {county}

# Populate district with their countries
for i, row in CEDA_data.iterrows():
  district = row['place']

  # Append counties
  if len(counties[district]) > 1:
    multi_county.append(1)
  else:
    multi_county.append(0)
  which_counties.append(counties[district])

# Add matched and county columns to CEDA data
CEDA_data['matched'] = matched
CEDA_data['matched to DOE'] = which_matched
CEDA_data['multi_county'] = multi_county
CEDA_data['which counties'] = which_counties

In [ ]:
CEDA_data.to_excel('/content/drive/MyDrive/Candidate Platforms and Endorsements/Data files/Unique district-year elections from CEDA with DOE matches.xlsx', index=False)

In [ ]:
CEDA_data[CEDA_data['matched to DOE']==0]

,cntyname,year,place,election_type,countyboard,coe,leaid,matched,matched to DOE,multi_county,which counties


## 1.3 Match from CEDA to DOE

In [ ]:
# Load Department of Education data
DOE_data = pd.read_excel('/content/drive/My Drive/Candidate Platforms and Endorsements/Three files discussed 10-18-23/Base_File_from_State_Dept_Ed.xls')
DOE_data.head()

,County,Year,DOCType,District,Status,WebSite,cds7,leaid,DOC
0,Alameda,1996,Unified School District,Alameda Unified,Active,http://www.alamedaunified.org,161119,601770,54
1,Alameda,1997,Unified School District,Alameda Unified,Active,http://www.alamedaunified.org,161119,601770,54
2,Alameda,1998,Unified School District,Alameda Unified,Active,http://www.alamedaunified.org,161119,601770,54
3,Alameda,1999,Unified School District,Alameda Unified,Active,http://www.alamedaunified.org,161119,601770,54
4,Alameda,2000,Unified School District,Alameda Unified,Active,http://www.alamedaunified.org,161119,601770,54


In [ ]:
# Load truncated unique district-year CEDA elections
CEDA_data = pd.read_excel('/content/drive/My Drive/Candidate Platforms and Endorsements/Three files discussed 10-18-23/All unique district-year elections from CEDA.xlsx')
CEDA_data.head()

,CNTYNAME,YEAR,PLACE,election_type,countyboard,coe,leaid
0,ALAMEDA,1996,ALAMEDA BOARD OF EDUCATION,Regular,NaN,NaN,NaN
1,ALAMEDA,2000,ALAMEDA CITY UNIFIED,Regular,NaN,NaN,NaN
2,ALAMEDA,2004,ALAMEDA CITY UNIFIED,Regular,NaN,NaN,NaN
3,ALAMEDA,2006,ALAMEDA CITY UNIFIED,Regular,NaN,NaN,NaN
4,ALAMEDA,2010,ALAMEDA CITY UNIFIED,Regular,NaN,NaN,NaN


In [ ]:
CEDA_data = CEDA_data[(CEDA_data['countyboard'] != 1) & (CEDA_data['coe'] != 1)]
CEDA_data = CEDA_data.drop(['countyboard', 'coe'], axis=1)

In [ ]:
# Standardize column names
CEDA_data.columns = [x.lower() for x in CEDA_data.columns]
DOE_data.columns = [x.lower() for x in DOE_data.columns]

# Standardize strings
CEDA_data['cntyname'] = CEDA_data['cntyname'].str.lower()
CEDA_data['place'] = CEDA_data['place'].str.lower()

DOE_data['county'] = DOE_data['county'].str.lower()
DOE_data['district'] = DOE_data['district'].str.lower()

In [ ]:
matches = []

# Keep matches in dictionary also
DOE_matched = {}

# Keep track of leaid
leaids = []

# Iterate through CEDA data
for i, row in CEDA_data.iterrows():
  year = row['year']
  county = row['cntyname']
  district = row['place']
  candidates = list(DOE_data[(DOE_data['year'] == year) & (DOE_data['county'] == county)]['district'])

  # Dictionary for filtered: original name
  cand_dict = {}

  # Extract keywords in district name
  words = district.split()
  distr_word_list = [words[0]]
  for i in range(1, len(words)):
    if words[i] not in filler:
      distr_word_list.append(words[i])

  # Join keywords in district
  cleaned_distr = " ".join(distr_word_list)

  # Extract keywords in candidates
  filtered = []
  for candidate in candidates:
    words = candidate.split()
    cand_word_list = []

    # Extract keywords in candidates
    for i in range(len(words)):
      if words[i] not in filler:
        cand_word_list.append(words[i])

    # Join keywords in candidates
    cleaned_cand = ' '.join(cand_word_list)

    # Cleaned: original candidate
    cand_dict[cleaned_cand] = candidate

    # Consider candidate only if have common keyword
    for keyword in distr_word_list:
      if keyword in cand_word_list:
        filtered.append(cleaned_cand)
        break
  candidates = filtered

  if len(candidates) == 0:
    matches.append(['none', 0])
    leaids.append(-1)
  else:
    match, score = process.extractOne(district, candidates, scorer=fuzz.token_set_ratio)
    matches.append([cand_dict[match], score])
    leaids.append(DOE_data[(DOE_data['year'] == year) & (DOE_data['county'] == county) & (DOE_data['district'] == cand_dict[match])]['leaid'].item())
    DOE_matched[(district, year, county)] = cand_dict[match]


In [ ]:
# Append matches into columns  for elem in matches]
CEDA_data['match'] = [elem[0] for elem in matches]
CEDA_data['score'] = [elem[1] for elem in matches]
CEDA_data['multi_county'] = multi_county
CEDA_data['which counties'] = which_counties
CEDA_data['leaid'] = leaids
CEDA_data

,cntyname,year,place,election_type,leaid,match,score,multi_county,which counties
0,alameda,1996,alameda board of education,Regular,601770,alameda unified,100,0,{alameda}
1,alameda,2000,alameda city unified,Regular,601770,alameda unified,100,0,{alameda}
2,alameda,2004,alameda city unified,Regular,601770,alameda unified,100,0,{alameda}
3,alameda,2006,alameda city unified,Regular,601770,alameda unified,100,0,{alameda}
4,alameda,2010,alameda city unified,Regular,601770,alameda unified,100,0,{alameda}
...,...,...,...,...,...,...,...,...,...
8357,yuba,2012,wheatland union high,Regular,642350,wheatland union high,100,0,{yuba}
8358,yuba,2014,wheatland union high,Regular,642350,wheatland union high,100,0,{yuba}
8359,yuba,2016,wheatland union high,Regular,642350,wheatland union high,100,0,{yuba}
8360,yuba,2020,wheatland union high,Regular,642350,wheatland union high,100,0,{yuba}


In [ ]:
CEDA_data.to_excel('/content/drive/MyDrive/Candidate Platforms and Endorsements/Data files/merged_CEDA-DOE.xlsx', index=False)

## 1.4 Merge final candidate + CEDA districts with leaid data set

In [ ]:
# Get CEDA data with leaid merged from DOE
district_data = pd.read_excel('/content/drive/MyDrive/Candidate Platforms and Endorsements/Data files/merged_CEDA-DOE.xlsx')
district_data

,cntyname,year,place,election_type,leaid,match,score,multi_county,which counties
0,alameda,1996,alameda board of education,Regular,601770,alameda unified,100,0,{'alameda'}
1,alameda,2000,alameda city unified,Regular,601770,alameda unified,100,0,{'alameda'}
2,alameda,2004,alameda city unified,Regular,601770,alameda unified,100,0,{'alameda'}
3,alameda,2006,alameda city unified,Regular,601770,alameda unified,100,0,{'alameda'}
4,alameda,2010,alameda city unified,Regular,601770,alameda unified,100,0,{'alameda'}
...,...,...,...,...,...,...,...,...,...
7868,yuba,2012,wheatland union high,Regular,642350,wheatland union high,100,0,{'yuba'}
7869,yuba,2014,wheatland union high,Regular,642350,wheatland union high,100,0,{'yuba'}
7870,yuba,2016,wheatland union high,Regular,642350,wheatland union high,100,0,{'yuba'}
7871,yuba,2020,wheatland union high,Regular,642350,wheatland union high,100,0,{'yuba'}


In [ ]:
# Get CEDA data with candidates
cand_data = pd.read_excel('/content/drive/MyDrive/Candidate Platforms and Endorsements/Data files/appended_CEDA_w_priorities.xlsx')

# Lowercase everything
cand_data['cntyname'] = cand_data['cntyname'].str.lower()
cand_data['place'] = cand_data['place'].str.lower()
cand_data['last'] = cand_data['last'].str.lower()
cand_data['first'] = cand_data['first'].str.lower()
cand_data['baldesig'] = cand_data['baldesig'].str.lower()
cand_data['incumb'] = cand_data['incumb'].str.lower()

cand_data

,cntyname,year,place,last,first,baldesig,office,incumb,co#,multi_candid,multi_co,raceid,percent,elected,newelected,priorities
0,alameda,1996,alameda board of education,greely,gail ann,incumbent and parent,BOARD MEMBER,y,1,199603002,0,199600920,0.208098,1,1,NaN
1,alameda,1996,alameda board of education,guenther,barbara m.,parent/case manager/advocate,BOARD MEMBER,n,1,199603003,0,199600920,0.180278,1,1,NaN
2,alameda,1996,alameda board of education,huie,sam,grandparent/computer professional,BOARD MEMBER,n,1,199603004,0,199600920,0.147389,2,2,NaN
3,alameda,1996,alameda board of education,linebarry,claude s. butch,management consultant,BOARD MEMBER,n,1,199603005,0,199600920,0.067111,2,2,NaN
4,alameda,1996,alameda board of education,mcmahon,michael,systems analyst,BOARD MEMBER,n,1,199603006,0,199600920,0.099790,2,2,Securing sufficient and stable funding for Ala...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45179,yuba,2016,yuba county office of education,hastey,desiree,small business owner,SCHOOL BOARD MEMBER,n,58,201605067,0,1630,0.466427,1,1,NaN
45180,yuba,2016,yuba county office of education,bishop,tracy,youth program representative,SCHOOL BOARD MEMBER,n,58,201605068,0,1630,0.358610,2,2,NaN
45181,yuba,2016,yuba county office of education,dawson,emily torres,incumbent,SCHOOL BOARD MEMBER,y,58,201605069,0,1630,0.174963,2,2,NaN
45182,yuba,2020,yuba county office of education,myerpeter-newman,anna,retired teacher,SCHOOL BOARD MEMBER,n,58,202006601,0,2209,0.461054,2,2,NaN


In [ ]:
# Get all (year, county, district) in districts data
districts = set()

for i, row in district_data.iterrows():
  districts.add((row['year'], row['cntyname'], row['place']))

# Only get candidates that having matching (year, county, district) in CEDA districts data
for i, row in cand_data.iterrows():
  if (row['year'], row['cntyname'], row['place']) not in districts:
    cand_data = cand_data.drop(i)

cand_data

,cntyname,year,place,last,first,baldesig,office,incumb,co#,multi_candid,multi_co,raceid,percent,elected,newelected,priorities
0,alameda,1996,alameda board of education,greely,gail ann,incumbent and parent,BOARD MEMBER,y,1,199603002,0,199600920,0.208098,1,1,NaN
1,alameda,1996,alameda board of education,guenther,barbara m.,parent/case manager/advocate,BOARD MEMBER,n,1,199603003,0,199600920,0.180278,1,1,NaN
2,alameda,1996,alameda board of education,huie,sam,grandparent/computer professional,BOARD MEMBER,n,1,199603004,0,199600920,0.147389,2,2,NaN
3,alameda,1996,alameda board of education,linebarry,claude s. butch,management consultant,BOARD MEMBER,n,1,199603005,0,199600920,0.067111,2,2,NaN
4,alameda,1996,alameda board of education,mcmahon,michael,systems analyst,BOARD MEMBER,n,1,199603006,0,199600920,0.099790,2,2,Securing sufficient and stable funding for Ala...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45162,yuba,2020,wheatland union high,forest,greg,no ballot designation,SCHOOL BOARD MEMBER,n,58,202006597,0,2208,0.260012,1,1,NaN
45163,yuba,2020,wheatland union high,mchugh,brendan,no ballot designation,SCHOOL BOARD MEMBER,n,58,202006598,0,2208,0.205447,2,2,NaN
45164,yuba,2020,wheatland union high,meder,shawndel,registered nurse,SCHOOL BOARD MEMBER,y,58,202006599,0,2208,0.317681,1,1,NaN
45165,yuba,1998,yuba co. office of education,anderson-chappel,cleo,technical support,SCHOOL BOARD MEMBER,n,58,199805181,0,199801741,0.321562,2,2,NaN


In [ ]:
# Get leaid into candidates from district
leaids = []

# Get matches into candidates from district
matches = []

for i, row in cand_data.iterrows():
  year = row['year']
  county = row['cntyname']
  district = row['place']

  leaids.append(district_data[(district_data['year'] == year) & (district_data['cntyname'] == county) & (district_data['place'] == district)]['leaid'].item())
  matches.append(district_data[(district_data['year'] == year) & (district_data['cntyname'] == county) & (district_data['place'] == district)]['match'].item())

cand_data['leaid'] = leaids
cand_data['matches'] = matches

In [ ]:
cand_data

,cntyname,year,place,last,first,baldesig,office,incumb,co#,multi_candid,multi_co,raceid,percent,elected,newelected,priorities,leaid,matches
0,alameda,1996,alameda board of education,greely,gail ann,incumbent and parent,BOARD MEMBER,y,1,199603002,0,199600920,0.208098,1,1,NaN,601770,alameda unified
1,alameda,1996,alameda board of education,guenther,barbara m.,parent/case manager/advocate,BOARD MEMBER,n,1,199603003,0,199600920,0.180278,1,1,NaN,601770,alameda unified
2,alameda,1996,alameda board of education,huie,sam,grandparent/computer professional,BOARD MEMBER,n,1,199603004,0,199600920,0.147389,2,2,NaN,601770,alameda unified
3,alameda,1996,alameda board of education,linebarry,claude s. butch,management consultant,BOARD MEMBER,n,1,199603005,0,199600920,0.067111,2,2,NaN,601770,alameda unified
4,alameda,1996,alameda board of education,mcmahon,michael,systems analyst,BOARD MEMBER,n,1,199603006,0,199600920,0.099790,2,2,Securing sufficient and stable funding for Ala...,601770,alameda unified
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45162,yuba,2020,wheatland union high,forest,greg,no ballot designation,SCHOOL BOARD MEMBER,n,58,202006597,0,2208,0.260012,1,1,NaN,642350,wheatland union high
45163,yuba,2020,wheatland union high,mchugh,brendan,no ballot designation,SCHOOL BOARD MEMBER,n,58,202006598,0,2208,0.205447,2,2,NaN,642350,wheatland union high
45164,yuba,2020,wheatland union high,meder,shawndel,registered nurse,SCHOOL BOARD MEMBER,y,58,202006599,0,2208,0.317681,1,1,NaN,642350,wheatland union high
45165,yuba,1998,yuba co. office of education,anderson-chappel,cleo,technical support,SCHOOL BOARD MEMBER,n,58,199805181,0,199801741,0.321562,2,2,NaN,-1,none


In [ ]:
cand_data.to_excel('/content/drive/MyDrive/Candidate Platforms and Endorsements/Data files/final_candidate_data.xlsx', index=False)

# 1.5 Merge School District Data from Urban Institute API

In [ ]:
'''ccd_columns = ['lea_name', 'fips', 'state_leaid', 'street_mailing', 'city_mailing', 'state_mailing', 'zip_mailing', 'zip4_mailing', 'street_location', 'city_location', 'state_location', 'zip_location', 'zip4_location', 'phone', 'latitude', 'longitude', 'urban_centric_locale', 'cbsa', 'cbsa_type', 'csa', 'cmsa', 'necta', 'county_code', 'county_name', 'congress_district_id', 'bureau_indian_education', 'supervisory_union_number', 'agency_type', 'agency_level', 'boundary_change_indicator', 'agency_charter_indicator', 'lowest_grade_offered', 'highest_grade_offered', 'number_of_schools', 'enrollment', 'spec_ed_students', 'english_language_learners', 'migrant_students', 'teachers_prek_fte', 'teachers_kindergarten_fte', 'teachers_elementary_fte', 'teachers_secondary_fte', 'teachers_ungraded_fte', 'teachers_total_fte', 'instructional_aides_fte', 'coordinators_fte', 'guidance_counselors_elem_fte', 'guidance_counselors_sec_fte', 'guidance_counselors_other_fte', 'guidance_counselors_total_fte', 'librarian_specialists_fte', 'librarian_support_staff_fte', 'lea_administrators_fte', 'lea_admin_support_staff_fte', 'school_administrators_fte', 'school_admin_support_staff_fte', 'support_staff_students_fte', 'support_staff_other_fte', 'staff_total_fte', 'lea_staff_total_fte', 'other_staff_fte', 'school_staff_total_fte', 'school_counselors_fte', 'state_leg_district_lower', 'state_leg_district_upper', 'school_psychologists_fte', 'support_staff_stu_wo_psych_fte']
saipe_columns = ['district_id', 'district_name', 'est_population_total', 'est_population_5_17', 'est_population_5_17_poverty', 'fips', 'est_population_5_17_poverty_pct', 'est_population_5_17_pct']

for col in ccd_columns:
  cand_data[col] = None  # You can assign default values or None if needed

for col in saipe_columns:
  cand_data[col] = None  # You can assign default values or None if needed

cand_data
'''

,cntyname,year,place,last,first,baldesig,office,incumb,co#,multi_candid,...,state_leg_district_upper,school_psychologists_fte,support_staff_stu_wo_psych_fte,district_id,district_name,est_population_total,est_population_5_17,est_population_5_17_poverty,est_population_5_17_poverty_pct,est_population_5_17_pct
0,alameda,1996,alameda board of education,greely,gail ann,incumbent and parent,BOARD MEMBER,y,1,199603002,...,None,None,None,None,None,None,None,None,None,None
1,alameda,1996,alameda board of education,guenther,barbara m.,parent/case manager/advocate,BOARD MEMBER,n,1,199603003,...,None,None,None,None,None,None,None,None,None,None
2,alameda,1996,alameda board of education,huie,sam,grandparent/computer professional,BOARD MEMBER,n,1,199603004,...,None,None,None,None,None,None,None,None,None,None
3,alameda,1996,alameda board of education,linebarry,claude s. butch,management consultant,BOARD MEMBER,n,1,199603005,...,None,None,None,None,None,None,None,None,None,None
4,alameda,1996,alameda board of education,mcmahon,michael,systems analyst,BOARD MEMBER,n,1,199603006,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45162,yuba,2020,wheatland union high,forest,greg,no ballot designation,SCHOOL BOARD MEMBER,n,58,202006597,...,None,None,None,None,None,None,None,None,None,None
45163,yuba,2020,wheatland union high,mchugh,brendan,no ballot designation,SCHOOL BOARD MEMBER,n,58,202006598,...,None,None,None,None,None,None,None,None,None,None
45164,yuba,2020,wheatland union high,meder,shawndel,registered nurse,SCHOOL BOARD MEMBER,y,58,202006599,...,None,None,None,None,None,None,None,None,None,None
45165,yuba,1998,yuba co. office of education,anderson-chappel,cleo,technical support,SCHOOL BOARD MEMBER,n,58,199805181,...,None,None,None,None,None,None,None,None,None,None


In [ ]:
'''
from pprint import pprint
import requests

ccd_url = "https://educationdata.urban.org/api/v1/school-districts/ccd/directory/"
saipe_url = "https://educationdata.urban.org/api/v1/school-districts/saipe/"
keys_to_remove = ['year', 'leaid']


i = 0
for index, row in cand_data.iterrows():
  leaid = "0" + str(row['leaid'])
  leaid = "123123312312"
  year = str(row['year'])
  curr_ccd_url = ccd_url + year
  curr_saipe_url = saipe_url + year

  payload = {
    "leaid": leaid
  }

  # Pass filters into API request

  ccd_response = requests.get(curr_ccd_url, params=payload).json()['results']
  saipe_response = requests.get(curr_saipe_url, params=payload).json()['results']

  if len(saipe_response) != 0:
    saipe_response = saipe_response[0]
    for key in keys_to_remove:
      saipe_response.pop(key, None)

      # add the json data to the cand_data df
      for key, value in saipe_response.items():
        row[key] = value


  if len(ccd_response) != 0:
    print("Found ccd data")
    print(ccd_response)
    ccd_response = ccd_response[0]
    for key in keys_to_remove:
      ccd_response.pop(key, None)

      # add the json data to the cand_data df
      for key, value in ccd_response.items():
        row[key] = value
  i += 1
  if i % 100 == 0: break


cand_data
'''

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


KeyboardInterrupt: ignored

# 1.6 Append priorities features into final dataset

In [ ]:
!pip install fuzzywuzzy

In [ ]:
cand_labels = pd.read_excel('/content/drive/My Drive/Candidate Platforms and Endorsements/Data files/Priorities_Latest_10-10-23_with_Ten_Topics.xlsx')

In [ ]:
# Drop NaN
cand_labels = cand_labels.dropna()

# Standardize columns
cand_labels.columns = [x.lower() for x in cand_labels.columns]

# Lowercase year and name
cand_labels['candidate_name'] = cand_labels['candidate_name'].str.lower()
cand_labels['districtname'] = cand_labels['districtname'].str.lower()
cand_labels['county'] = cand_labels['county'].str.lower()
cand_labels['priority'] = cand_labels['priority'].str.lower()

<ipython-input-9-e5f9423795ac>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cand_labels['candidate_name'] = cand_labels['candidate_name'].str.lower()
<ipython-input-9-e5f9423795ac>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cand_labels['districtname'] = cand_labels['districtname'].str.lower()
<ipython-input-9-e5f9423795ac>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [ ]:
cand_labels.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30154 entries, 0 to 30179
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   year            30154 non-null  int64  
 1   candidate_name  30154 non-null  object 
 2   districtname    30154 non-null  object 
 3   county          30154 non-null  object 
 4   priority        30154 non-null  object 
 5   topic_0         30154 non-null  float64
 6   topic_1         30154 non-null  float64
 7   topic_2         30154 non-null  float64
 8   topic_3         30154 non-null  float64
 9   topic_4         30154 non-null  float64
 10  topic_5         30154 non-null  float64
 11  topic_6         30154 non-null  float64
 12  topic_7         30154 non-null  float64
 13  topic_8         30154 non-null  float64
 14  topic_9         30154 non-null  float64
dtypes: float64(10), int64(1), object(4)
memory usage: 3.7+ MB


In [ ]:
CEDA_priorities = pd.read_excel('/content/drive/My Drive/Candidate Platforms and Endorsements/Data files/merged_final_candidate_data.xlsx')

In [ ]:
cand_labels

,year,candidate_name,districtname,county,priority,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9
0,1997,suzan mannisto,belmont-redwood shores elementary school district,san mateo,manage growth of district as flow of students ...,0.037900,0.046479,0.028348,0.044122,0.027025,0.132960,0.168425,0.231308,0.033500,0.249934
1,1997,suzan mannisto,belmont-redwood shores elementary school district,san mateo,maintain high quality of curricula with input ...,0.146378,0.105995,0.026576,0.041366,0.025336,0.076238,0.220314,0.279256,0.031407,0.047134
2,1997,suzan mannisto,belmont-redwood shores elementary school district,san mateo,"encourage individual ""personality"" of each sch...",0.160340,0.105995,0.026577,0.041367,0.092849,0.181995,0.157896,0.154439,0.031407,0.047135
3,1997,catherine bonnar,belmont-redwood shores elementary school district,san mateo,expand sandpiper school to a full k-5th grade ...,0.037942,0.111998,0.028379,0.044171,0.027054,0.199776,0.101958,0.231547,0.166846,0.050330
4,1997,catherine bonnar,belmont-redwood shores elementary school district,san mateo,wisely budget $12 million bond revenue with co...,0.031591,0.204871,0.023629,0.092274,0.022881,0.055342,0.251300,0.137309,0.083413,0.097390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30175,2022,rogelio villagrana,"trustee area 7, woodland joint unified school ...",yolo,student involvement in the decision making pro...,0.062719,0.027904,0.017018,0.026489,0.016224,0.119759,0.396327,0.243302,0.020111,0.070147
30176,2022,rogelio villagrana,"trustee area 7, woodland joint unified school ...",yolo,maximize community resources to increase suppo...,0.022754,0.067863,0.017019,0.227361,0.016225,0.159773,0.179999,0.178841,0.060029,0.070135
30177,2022,catherine saeturn,"trustee area 7, woodland joint unified school ...",yolo,prioritize communication and transparency by a...,0.043718,0.130405,0.032699,0.050896,0.035209,0.076586,0.117480,0.339581,0.115432,0.057993
30178,2022,catherine saeturn,"trustee area 7, woodland joint unified school ...",yolo,set goals and create opportunities for our com...,0.043718,0.053615,0.109487,0.050896,0.031173,0.076585,0.208120,0.329771,0.038642,0.057992


In [ ]:
CEDA_priorities

,cntyname,year,place,last,first,baldesig,office,incumb,co#,multi_candid,...,state_leg_district_upper,school_psychologists_fte,support_staff_stu_wo_psych_fte,district_id,district_name,est_population_total,est_population_5_17,est_population_5_17_poverty,est_population_5_17_poverty_pct,est_population_5_17_pct
0,alameda,1996,alameda board of education,greely,gail ann,incumbent and parent,BOARD MEMBER,y,1,199603002,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,alameda,1996,alameda board of education,guenther,barbara m.,parent/case manager/advocate,BOARD MEMBER,n,1,199603003,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,alameda,1996,alameda board of education,huie,sam,grandparent/computer professional,BOARD MEMBER,n,1,199603004,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,alameda,1996,alameda board of education,linebarry,claude s. butch,management consultant,BOARD MEMBER,n,1,199603005,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,alameda,1996,alameda board of education,mcmahon,michael,systems analyst,BOARD MEMBER,n,1,199603006,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37725,yuba,2020,wheatland union high,forest,greg,no ballot designation,SCHOOL BOARD MEMBER,n,58,202006597,...,6004.0,0.0,1.0,42350.0,Wheatland Union High School District,14717.0,832.0,60.0,0.072115,0.056533
37726,yuba,2020,wheatland union high,mchugh,brendan,no ballot designation,SCHOOL BOARD MEMBER,n,58,202006598,...,6004.0,0.0,1.0,42350.0,Wheatland Union High School District,14717.0,832.0,60.0,0.072115,0.056533
37727,yuba,2020,wheatland union high,meder,shawndel,registered nurse,SCHOOL BOARD MEMBER,y,58,202006599,...,6004.0,0.0,1.0,42350.0,Wheatland Union High School District,14717.0,832.0,60.0,0.072115,0.056533
37728,yuba,1998,yuba co. office of education,anderson-chappel,cleo,technical support,SCHOOL BOARD MEMBER,n,58,199805181,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

def fuzzy_match_and_merge(candidate_data, ceda_data):
    # Function to find the best fuzzy match for each candidate in the CEDA dataset
    def find_best_match(name, candidates):
        match, score,_ = process.extractOne(name, candidates, scorer=fuzz.token_set_ratio)
        return match, score

    # Create a dictionary of candidate names as keys and their priorities as values
    name_to_labels = {}
    name_to_score = {}

    num_added = 0
    num_read = 0

    for i, row in ceda_data.iterrows():
        ceda_name = f"{row['first']} {row['last']}"
        ceda_year = row['year']
        ceda_county = row['cntyname']

        # Filter candidate data to the same year and county
        year_county_matched_candidates = candidate_data[(candidate_data['year'] == ceda_year) & (candidate_data['county'] == ceda_county)]

        if not year_county_matched_candidates.empty:
            match, score = find_best_match(ceda_name, year_county_matched_candidates['candidate_name'])
            if score > 80:
                # Found a match with a high enough score, add to dictionary
                matched_rows = year_county_matched_candidates.loc[
                    year_county_matched_candidates['candidate_name'] == match
                ]

                for topic_col in ['topic_0', 'topic_1', 'topic_2', 'topic_3', 'topic_4', 'topic_5', 'topic_6', 'topic_7', 'topic_8', 'topic_9']:
                  topic_vals = matched_rows[topic_col].values
                  ceda_data.at[i, topic_col] = np.mean(topic_vals)

                  if (ceda_name, ceda_year, ceda_county) in name_to_labels:
                    name_to_labels[(ceda_name, ceda_year, ceda_county)].append(np.mean(topic_vals))
                  else:
                    name_to_labels[(ceda_name, ceda_year, ceda_county)] = [np.mean(topic_vals)]

                name_to_score[ceda_name] = score  # Store the score
                num_added += 1

                #if score != 100:
                #  print("Name in CEDA data: ", ceda_name)
                #  print("Matched name from candidate data: ", match)
                #  print("Year: ", ceda_year)
                #  print("Score was: ", score)
                #  print(row)

        num_read += 1

        #if num_read % 100 == 0:
        #  print(num_added , "samples have been matched out of ", num_read)

    return ceda_data, name_to_labels, name_to_score

In [ ]:
CEDA_priorities, name_to_labels, name_to_score = fuzzy_match_and_merge(cand_labels, CEDA_priorities)

# Truncate keys where there are duplicates in CEDA (ie. Forrest Gee registered twice in 2001 Alameda for some reason)
for key in name_to_labels:
  if len(name_to_labels[key]) > 10:
    name_to_labels[key] = name_to_labels[key][:10]

In [ ]:
'''
    # Create a new 'score' column in the CEDA dataset
    ceda_data['score'] = ceda_data.apply(
        lambda row: name_to_score.get(f"{row['first']} {row['last']}", None), axis=1
    )

    # Create 10 topic columns
    ceda_data = pd.concat([ceda_data, pd.DataFrame(name_to_labels)], axis=1)
'''

In [ ]:
CEDA_priorities.to_excel('/content/drive/MyDrive/Candidate Platforms and Endorsements/Data files/featurized_CEDA_priorities.xlsx', index=False)